In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import solve_ivp
from scipy import interpolate

In [3]:
# Agrega el directorio superior al path
sys.path.append(os.path.abspath('..'))

In [4]:
import stars.EDOs.Sistemas as eds
import stars.Utilez as eu

In [5]:
eu.plots.general()

#### Veamos un ejemplo de resolución del sistema EDO para una estrella 

#### Veamos un ejemplo de resolución del sistema EDO usando una EoS realista